In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
clauses = []

num_literals = 0
num_clause = 0

with open('../UF250.1065.100/uf250-01.cnf') as file:
    for line in file:
        if(line[0] == 'p'):
            l = line.split()
            num_literals = int(l[2])
            num_clauses = int(l[3])
        if(line[0] != 'p' and line[0] != '%' and line[0] != 'c'):
            clause = [int(digit) for digit in line.split()]
            clauses.append(clause[:-1])

In [3]:
from numpy import linalg as LA

In [4]:
import math

In [15]:
def rbf(xi, xj, h):
    return math.exp(-((LA.norm(xi - xj))**2) / h)

In [33]:
def rbf_derivative(xi, xj, h, k):
    return (2 / h) * k * (xi - xj)

In [34]:
#returns the likelihood of each clause being F for each sample
def likelihood_clauses():
    likelihoods = np.zeros((num_points, num_clauses))
    for i in range(num_points):
        for j in range(num_clauses):
            p = 1
            for literal in clauses[j]:
                if literal > 0:
                    p = p * (1 - points[i][literal-1])
                else:
                    p = p * points[i][-literal-1]
            likelihoods[i][j] = p
    return likelihoods       

In [35]:
#return the derivative for the i-th sample
def derivative_likelihood(alpha, beta, i, likelihoods):
    derivative = np.zeros(num_literals)
    for k in range(num_literals):
        derivative[k] += ((alpha - 1) / points[i][k]) - ((beta - 1) / (1 - points[i][k]))
        
    for c in range(num_clauses):
        likelihood = likelihoods[i][c]
        for literal in clause:
            if literal > 0:
                derivative[literal-1] += (likelihood / ( 1- points[i][literal-1])) / (1 - likelihood)
            else:
                derivative[-literal-1] += - (likelihood / points[i][-literal-1]) / (1 - likelihood)
                
    return derivative
    

In [36]:
def satisfied(polarity):
    num_satisfied = 0
    for clause in clauses:
        for literal in clause:
            if literal * polarity[abs(literal) - 1] > 0:
                num_satisfied += 1
                break
    return num_satisfied

In [37]:
num_points = 20
points = np.random.beta(2, 2, size = (num_points, num_literals))
def stein_update(num_epochs):
    satisfied_clauses = np.zeros(num_epochs)
    for e in range(num_epochs):
        ave = np.sum(points, axis=0) / num_points
        polarity = np.zeros(num_literals)
        for i in range(num_literals):
            if ave[i] > 0.5:
                polarity[i] = 1
            else:
                polarity[i] = -1
        satisfied_clauses[e] = satisfied(polarity)
        kernel_matrix = np.zeros((num_points, num_points))
        h = 1
        for i in range(num_points):
            for j in range(num_points):
                if i <= j:
                    kernel_matrix[i][j] = rbf(points[i], points[j], h)
                else:
                    kernel_matrix[i][j] = kernel_matrix[j][i]  
        new_points = np.zeros((num_points, num_literals))
        likelihoods = likelihood_clauses()
        derivatives_likelihood = np.zeros((num_points, num_literals))
        for i in range(num_points):
            derivatives_likelihood[i] = derivative_likelihood(0.5, 0.5, i, likelihoods)
        epsilon = 0.01
        for i in range(num_points):
            phi = 0
            for j in range(num_points):
                phi += kernel_matrix[i][j] * derivatives_likelihood[j] + rbf_derivative(i, j, h, kernel_matrix[i][j])
            phi = phi / num_points
            new_points[i] = points[i] + epsilon * phi
        for i in range(num_points):
            for j in range(num_literals):
                if new_points[i][j] < 0.01:
                    points[i][j] = 0.01
                elif new_points[i][j] > 0.99:
                    points[i][j] = 0.99
                else:
                    points[i][j] = new_points[i][j]
    return satisfied_clauses

In [38]:
stein_update(30)

array([930., 930., 930., 930., 933., 933., 933., 933., 932., 929., 930.,
       930., 930., 930., 930., 930., 930., 932., 932., 932., 933., 934.,
       934., 930., 930., 930., 930., 930., 930., 930.])